In [ ]:
import os
lib_folder = os.path.abspath("../src/DerivaSharp/bin/Release/net10.0/win-x64/publish")

import pythonnet
pythonnet.load("coreclr", runtime_config=os.path.join(lib_folder, "DerivaSharp.runtimeconfig.json"))

import clr
clr.AddReference(os.path.join(lib_folder, "DerivaSharp.dll"))

import matplotlib.pyplot as plt
import numpy as np

from DerivaSharp.Instruments import Accumulator
from DerivaSharp.Models import BsmModelParameters
from DerivaSharp.PricingEngines import McAccumulatorEngine, PricingContext
from System import DateOnly

In [ ]:
strike = 0.9
ko = 1.05
quantity = 1.0
factor = 2.0
vol = 0.16
r = 0.02
q = 0.04
effectiveDate = DateOnly(2022, 1, 5)
expirationDate = DateOnly(2023, 1, 31)

valuationDate = DateOnly(2022, 1, 5)

option = Accumulator(strike, ko, quantity, factor, 0.0, effectiveDate, expirationDate)

model = BsmModelParameters(vol, r, q)
context = PricingContext[BsmModelParameters](model, 1.0, valuationDate)

mc_engine = McAccumulatorEngine(500000, True)

In [ ]:
spots = np.arange(0.5, 1.51, 0.01, dtype=np.float64)
values = -np.array(mc_engine.Values(option, context, spots))
deltas = -np.array(mc_engine.Deltas(option, context, spots))
gammas = -np.array(mc_engine.Gammas(option, context, spots))

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(8, 10), sharex=True)

axes[0].plot(spots, values, label="Value")
axes[0].set_ylabel("Value")
axes[0].grid(True, linestyle="--", alpha=0.5)

axes[1].plot(spots, deltas, label="Delta", color="orange")
axes[1].set_ylabel("Delta")
axes[1].grid(True, linestyle="--", alpha=0.5)

axes[2].plot(spots, gammas, label="Gamma", color="green")
axes[2].set_ylabel("Gamma")
axes[2].set_xlabel("Spot")
axes[2].grid(True, linestyle="--", alpha=0.5)

for ax in axes:
    ax.legend()

fig.tight_layout()
plt.show()